In [62]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [37]:
# Initialize df for team data
teams_df = pd.DataFrame(columns=['Team', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS'])
# Populate Team Data with web scraping
def get_team_data():
    teams = []
    W = []
    L = []
    T = []
    W_L = []
    PF = []
    PA = []
    PD = []
    MoV = []
    SoS = []
    SRS = []
    OSRS = []
    DSRS = []
    
    html_text = requests.get('https://www.pro-football-reference.com/years/2022/index.htm').text
    soup = BeautifulSoup(html_text, 'lxml')
    tables = soup.find_all('tbody')
    
    for table in tables:
        teams_data = table.find_all('tr', class_="")
        for team_data in teams_data:
            teams.append(team_data.find('a').text)
            W.append(team_data.find_all('td')[0].text)
            L.append(team_data.find_all('td')[1].text)
            T.append(team_data.find_all('td')[2].text)
            W_L.append(team_data.find_all('td')[3].text)
            PF.append(team_data.find_all('td')[4].text)
            PA.append(team_data.find_all('td')[5].text)
            PD.append(team_data.find_all('td')[6].text)
            MoV.append(team_data.find_all('td')[7].text)
            SoS.append(team_data.find_all('td')[8].text)
            SRS.append(team_data.find_all('td')[9].text)
            OSRS.append(team_data.find_all('td')[10].text)
            DSRS.append(team_data.find_all('td')[11].text)
    teams_df['Team'] = teams
    teams_df['W'] = W
    teams_df['L'] = L
    teams_df['T'] = T
    teams_df['W-L%'] = W_L
    teams_df['PF'] = PF
    teams_df['PA'] = PA
    teams_df['PD'] = PD
    teams_df['MoV'] = MoV
    teams_df['SoS'] = SoS
    teams_df['SRS'] = SRS
    teams_df['OSRS'] = OSRS
    teams_df['DSRS'] = DSRS
get_team_data()

In [38]:
teams_df_sort = teams_df.sort_values(by = 'Team').reset_index(drop=True)

In [39]:
# Initialize df for offense redzone data
off_red_df = pd.DataFrame(columns=['Team', 'off_red_per'])
# Populate offense redzone data with web scraping
def get_off_red_data():
    teams = []
    percent = []
    html_text = requests.get('https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct?date=2022-11-10').text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('tbody')
    for row in table.find_all('tr'):
        teams.append(row.find('a').text)
        percent.append(row.find_all('td')[2].text)
    off_red_df['Team'] = teams
    off_red_df['off_red_per'] = percent
get_off_red_data()

In [40]:
off_red_df_sort = off_red_df.sort_values(by = 'Team').reset_index(drop=True)

In [41]:
new_teams_df = pd.concat([teams_df_sort, off_red_df_sort['off_red_per']], axis=1)
new_teams_df['off_red_per'] = new_teams_df['off_red_per'].str.rstrip('%').astype('float') / 100.0

In [42]:
# Initialize Schedule df for 2022
schedule_df = pd.read_csv('spreadspoke_scores.csv').groupby('schedule_season').get_group(2022)
pd.set_option('display.max_columns', None)
schedule_df


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13232,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52,SoFi Stadium,False,72.0,0.0,NaN,DOME
13233,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,54,State Farm Stadium,False,NaN,NaN,NaN,NaN
13234,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44,Mercedes-Benz Stadium,False,NaN,NaN,NaN,NaN
13235,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,42,Bank of America Stadium,False,NaN,NaN,NaN,NaN
13236,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,38,Soldier Field,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13499,1/8/2023,2022,18,False,Philadelphia Eagles,NaN,NaN,New York Giants,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN
13500,1/8/2023,2022,18,False,Pittsburgh Steelers,NaN,NaN,Cleveland Browns,NaN,NaN,NaN,Acrisure Stadium,False,NaN,NaN,NaN,NaN
13501,1/8/2023,2022,18,False,San Francisco 49ers,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN
13502,1/8/2023,2022,18,False,Seattle Seahawks,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Lumen Field,False,NaN,NaN,NaN,NaN


In [43]:
# df = schedule_df.merge(new_teams_df, left_on='team_home', right_on='Team', how='left').merge(teams_df, left_on='team_away', right_on='Team', how='left')
df = schedule_df.join(new_teams_df.set_index('Team'), on='team_home').join(new_teams_df.set_index('Team'), on='team_away', rsuffix='_away')
pd.set_option('display.max_columns', None)
df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,off_red_per,W_away,L_away,T_away,W-L%_away,PF_away,PA_away,PD_away,MoV_away,SoS_away,SRS_away,OSRS_away,DSRS_away,off_red_per_away
13232,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52,SoFi Stadium,False,72.0,0.0,NaN,DOME,3,5,0,.375,131,173,-42,-5.3,0.9,-4.3,-4.1,-0.2,0.5000,6,2,0,.750,220,118,102,12.8,1.5,14.3,6.1,8.2,0.5357
13233,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,54,State Farm Stadium,False,NaN,NaN,NaN,NaN,3,6,0,.333,203,241,-38,-4.2,0.0,-4.2,-1.1,-3.1,0.5769,6,2,0,.750,243,189,54,6.8,-0.7,6.1,9.4,-3.3,0.7353
13234,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44,Mercedes-Benz Stadium,False,NaN,NaN,NaN,NaN,4,5,0,.444,217,225,-8,-0.9,-1.3,-2.1,1.5,-3.6,0.6207,3,6,0,.333,212,227,-15,-1.7,-0.6,-2.3,0.6,-2.9,0.5714
13235,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,42,Bank of America Stadium,False,NaN,NaN,NaN,NaN,2,7,0,.222,179,228,-49,-5.4,-0.7,-6.2,-2.0,-4.2,0.5455,3,5,0,.375,200,199,1,0.1,0.2,0.3,2.9,-2.5,0.6129
13236,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,38,Soldier Field,False,NaN,NaN,NaN,NaN,3,6,0,.333,187,216,-29,-3.2,0.7,-2.6,0.7,-3.3,0.5517,4,4,0,.500,176,147,29,3.6,-1.7,2.0,-1.4,3.3,0.5769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13499,1/8/2023,2022,18,False,Philadelphia Eagles,NaN,NaN,New York Giants,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN,8,0,0,1.000,225,135,90,11.3,-2.0,9.3,5.5,3.8,0.7000,6,2,0,.750,163,157,6,0.8,0.1,0.8,-1.9,2.8,0.4615
13500,1/8/2023,2022,18,False,Pittsburgh Steelers,NaN,NaN,Cleveland Browns,NaN,NaN,NaN,Acrisure Stadium,False,NaN,NaN,NaN,NaN,2,6,0,.250,120,197,-77,-9.6,5.4,-4.2,-3.8,-0.4,0.4762,3,5,0,.375,200,199,1,0.1,0.2,0.3,2.9,-2.5,0.6129
13501,1/8/2023,2022,18,False,San Francisco 49ers,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN,4,4,0,.500,176,147,29,3.6,-1.7,2.0,-1.4,3.3,0.5769,3,6,0,.333,203,241,-38,-4.2,0.0,-4.2,-1.1,-3.1,0.5769
13502,1/8/2023,2022,18,False,Seattle Seahawks,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Lumen Field,False,NaN,NaN,NaN,NaN,6,3,0,.667,241,220,21,2.3,-2.2,0.1,3.7,-3.6,0.4828,3,5,0,.375,131,173,-42,-5.3,0.9,-4.3,-4.1,-0.2,0.5000


In [44]:
# Get completed games
comp_games = df[df['score_home'].notna()]
# Get uncompleted games
uncomp_games = df[df['score_home'].isna()]

In [45]:
for index, row in comp_games.iterrows():
    if comp_games.loc[index, 'score_home'] > comp_games.loc[index, 'score_away']:
        comp_games.loc[index, 'Home_Winner'] = True
    else:
        comp_games.loc[index, 'Home_Winner'] = False
    # else:
    #     comp_games.at[index, 'Winner'] = 'Tie'


/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_16834/1157996926.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_games.loc[index, 'Home_Winner'] = False


In [46]:
comp_games.drop(schedule_df.iloc[0:0], axis=1, inplace=True)
uncomp_games.drop(schedule_df.iloc[0:0], axis=1, inplace=True)

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_16834/3359712527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_games.drop(schedule_df.iloc[0:0], axis=1, inplace=True)
/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_16834/3359712527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncomp_games.drop(schedule_df.iloc[0:0], axis=1, inplace=True)


In [47]:
comp_games.columns

Index(['W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS',
       'DSRS', 'off_red_per', 'W_away', 'L_away', 'T_away', 'W-L%_away',
       'PF_away', 'PA_away', 'PD_away', 'MoV_away', 'SoS_away', 'SRS_away',
       'OSRS_away', 'DSRS_away', 'off_red_per_away', 'Home_Winner'],
      dtype='object')

In [48]:
y = comp_games['Home_Winner'].astype(int)
comp_games_X = comp_games.copy()
comp_games_X.drop(['Home_Winner'], axis=1, inplace=True)
X = comp_games_X

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [49]:
DT = DecisionTreeClassifier()
LSVC = LinearSVC()
GND = GaussianNB()
RFC = RandomForestClassifier()
DT.fit(X_train,y_train)
LSVC.fit(X_train,y_train)
GND.fit(X_train,y_train)
RFC.fit(X_train,y_train)

/opt/homebrew/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier()

In [68]:
tree.export_graphviz(DT, out_file="DT-visualization.dot",
                     feature_names=comp_games_X.columns,
                     class_names=y_train.astype(str).unique(),
                     label='all',
                     rounded=True,
                     filled=True)

In [50]:
dt_pred = DT.predict(X_test)
lsvc_pred = LSVC.predict(X_test)
gnd_pred = GND.predict(X_test)
rfc_pred = RFC.predict(X_test)

In [55]:
print(accuracy_score(y_test,dt_pred))
print(f1_score(y_test,dt_pred))

0.6153846153846154
0.7058823529411765


In [56]:
print(accuracy_score(y_test,lsvc_pred))
print(f1_score(y_test,lsvc_pred))

0.6923076923076923
0.75


In [53]:
print(accuracy_score(y_test,gnd_pred))
print(f1_score(y_test,gnd_pred))

0.7692307692307693
0.823529411764706


In [57]:
print(accuracy_score(y_test,rfc_pred))
print(f1_score(y_test,rfc_pred))

0.5384615384615384
0.6666666666666666


In [39]:
# cross validation